# DEM(数値標高モデル)からの水系図作成
ここでは、数値標高モデルのデータから水系図の作成を行います。

以下の2つの方法を説明します。
- QGISを経由してGRASS GISのモジュールを利用する
- PythonからGRASS GISのモジュールを利用する

どちらも、GRASSの[r.watershed](https://grass.osgeo.org/grass78/manuals/r.watershed.html)というモジュールを利用して水系図を作成します。GRASS GISは独自のデータベース構造を持つため、はじめは仕組みを理解することが難しいかもしれませんが、ラスタデータやベクタデータを扱う多くのライブラリがあり、リモートセンシングやGISの解析に非常に有用なツールです。QGISを経由してGRASSのモジュールを利用することで、比較的簡易にそれらのツールを利用することができるようになります。また、Pythonを使う方法を利用すると、バッチ処理やほかのPython処理とスムーズに結合することができるようになります。

まずはQGISを使った方法を説明します。

事前に、[OSGeo4W](https://trac.osgeo.org/osgeo4w/)からネットワークインストーラーをダウンロードして、QGIS Desktop with GRASSが使えるようにしておいてください。インストールの設定は、エクスプレスインストールを選択し、すべてデフォルトの設定で大丈夫です。

データは笛吹川上流域のSRTM DEMデータを利用します。SRTMデータはWGS84の地理座標系で提供されています。事前にUTM座標系に変換したものを準備したので、[ここから](data/watershed/dem_fuefuki.tif)ダウンロードしてdataフォルダに保存してください。

## QGISを用いた方法
スタートメニューから、QGISを検索し、**QGIS Desktop with GRASS**を起動してください。**with GRASS表記がないものでは以下は動きません。**

![QGISの起動](images/ss_catchment_01_start_qgis.png)

ダウンロードした`dem_fuefuki.tif`をQGISにドラッグ＆ドロップしてデータを読み込みます。読み込まれたデータはLayersパネルに一覧が表示されます。Layersパネルが表示されていない場合、View → PanelsメニューからLayersをオンにしてください。また、同じメニューから、Processing Tool Boxのパネルも表示してください。

<video controls src="../_static/mv_watershed_load_dem.mp4">animation</video>

### GRASS GISモジュールの実行
Processing Toolboxから、GRASS GISやSAGA GISなどのほかのフリーのGISソフトが提供するツール群を利用することができます。Processing Toolboxの検索ボックスに"water"と入れて表示される"r.watershed"モジュールを起動してください。

![watershedモジュールの起動](images/ss_watershed_rwatershed.png)

モジュールに以下の設定を行い、Runを押して実行してください。記載のない項目はすべてデフォルトの値です。

- Elevation : dem_fuefuki [EPSG:32654]
- Minimum size of exterior watershed basin [optional] : 1000

上記のオプションで最小面積が$1000m^2$となるような集水域図が作成されます。
ログメッセージが表示後、モジュールの実行が完了します。モジュールウィンドウは閉じてください。

### 結果
Layersパネルに計算されたラスターデータが追加されます。

作成されたラスタについて、以下で説明します。図はDEMから作成した陰影図を重ね合わせています。

#### Unique label for each watershed basin
集水域を表します。各集水域に固有のIDが振られています。  
![集水域](images/watershed_basin.png)

#### Half-basins
河川の左右別の集水域を表します。下流から見て右の支流に偶数、左の支流にその1少ない奇数のIDが振られています。  
![半集水域](images/watershed_basin_half.png)


#### Number of cells that drain through each cell
各ピクセルより上流にあるピクセルの数を表します。ピクセルサイズと掛け合わせることで、各ピクセルにおける集水域の面積となります。入力DEMの範囲外から流入する地域については値が負になっています。

![ピクセル数](images/watershed_n_cell.png)

#### Stream segments
河川の位置を表します。各河川は集水域のIDに対応するID値を持ちます。GRASSの`r.to.vect`モジュールの入力として与えることで、ベクターデータ(shapefile)として出力できます。

![stream](images/watershed_stream_seg.png)

#### Drainage direction
各ピクセルでの斜面の方位を示しています。方位は以下の図に示す値で定義されています。([参考](https://idea.isnew.info/how-to-import-arcgis-flow-direction-into-grass-gis.html))

![流下方位](images/watershed_drainage_direction_map.png)
![方位定義](images/watershed_drainage_direction.png)

#### Stream power index a * tan(b)
各地点における水が流下する力を示します。各ピクセルにおける単位等高線あたりの集水面積を$\alpha$、各地点での傾斜角度を$\beta$としたとき、$\alpha \times tan(\beta)$と定義されるStream Power Index(SPI)と呼ばれる指標です。(Moore et al. 1991)

![SPI](images/watershed_stream_pi.png)

#### Topographic index ln(a / tan(b))
各地点における水が流下する力を示します。SPIの自然対数で定義されます。(Quinn et al. 1991)

![TCI](images/watershed_topo_indx.png)

#### Slope length and steepness (LS) factor for USLE
Revised Universal Soil Loss Equation (RUSLE)におけるLSの値(傾斜長および傾斜度)を示します。

![LS](images/watershed_slope_ls.png)

#### Slope steepness (S) factor for USLE
Revised Universal Soil Loss Equation (RUSLE)におけるSの値(傾斜度)を示します。

![S](images/watershed_slope_s.png)

#### 結果の保存
Layerウィンドウ内でレイヤの右側に![メモリアイコン](images/watershed_memory.png)表示があるレイヤはQGISの一時ファイルに保存されています。GRASSモジュールを実行する際に計算結果の保存先を指定しなかった場合、デフォルトで一時ファイルに保存されます。これらはQGISを終了した際に削除されてしまうので、QGISを閉じる前にGeoTiffファイルとして保存してください。保存方法は、保存したいレイヤを右クリック、Export -> Save as... で表示されるダイヤログにおいて、フォーマットと保存先フォルダを指定します。その他の設定項目はデフォルト値で大丈夫です。ラスタがファイルに保存され、QGISのレイヤにも追加されます。

以上でQGISを通じたGRASSモジュールによる集水域図を作成することができました。

## PythonからGRASSモジュールを利用する
GRASS GISは、独自のGRASSデータベースを作成し、一度ラスタやベクタデータをデータベースに登録したうえで、各種の計算を行います。計算された結果はデータベース上にあるため、GeotiffやShapefile等の一般的なファイルとして
QGISからGRASSモジュールを利用する場合、以下のプロセスが自動で実行されます。
- GRAS

ref: https://grasswiki.osgeo.org/wiki/Working_with_GRASS_without_starting_it_explicitly

以下のコマンドでGRASS GISをインストールします。15分ほどかかります。
```bash
sudo apt install grass
```

In [ ]:
!pip install grass-session

In [2]:
from grass_session import Session
from grass.script import core as gcore

raster_path = 'data/watershed/dem_fuefuki.tif'

# create a new location from raster file
with Session(gisdb='tmp_grass_db',
             location='fuefuki',
             mapset='PERMANENT',
             create_opts=raster_path):
    # import geotiff to PERMANENT mapset and change extent
    gcore.run_command('r.in.gdal',
                      flags='oe',
                      input=raster_path,
                      band=1,
                      output='dem',
                      overwrite=True)
    # Run r.watershed command
    gcore.run_command('r.watershed',
                      elevation='dem',
                      threshold=1000,
                      accumulation='accumulation',
                      drainage='drainage',
                      basin='basin',
                      stream='stream',
                      half_basin='half_basin',
                      length_slope='length_slope',
                      slope_steepness='slope_steepness',
                      tci='tci',
                      spi='spi',
                      overwrite=True)
    # List up created rasters
    print(gcore.read_command('g.list',
                             type='rast'))


accumulation
basin
dem
drainage
half_basin
length_slope
slope_steepness
spi
stream
tci



In [3]:
with Session(gisdb='tmp_grass_db',
             location='fuefuki',
             mapset='PERMANENT'):
    gcore.run_command('r.out.gdal',
                      flags='mt',
                      input='accumulation',
                      output='accumulation.tif',
                      format='GTiff',
                      createopt='TFW=YES,COMPRESS=LZW',
                      overwrite=True)
    